In [7]:
import pandas as pd
import numpy as np
import datetime as dt
import calendar
from dateutil.relativedelta import relativedelta
import math
import re
import pyodbc



# Reading Source Data 'ADT_Dummy Data' and 'ADT_ER_IP_04012021to07062021'

server = 'wn000038725' 
database = 'oas_mas_stg' 
username = 'svcoimas'
password = 'Canopyanalytics#124'

#cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

df = pd.read_excel(r'C:\Users\ssing296\Desktop\Canopy\Data Files\ADT_ED_CSV_Py1.xlsx')

df['Diagnosis'].fillna('blank_code', inplace=True)

qry_insert = "Insert into  [Canopy].[ARCH_ADT_ANDOR_ED_input] (ExternalVisitId,MemberID,FirstName,LastName,Gender,DOB,PhoneNumber,LSPhoneNumber,MedicalGroup,AdmitDate,DischargeDate,PatientClass,EncounterType,SiteName,LocationName,Diagnosis,AdmittedInpatient,IPVisitCount1Month,IPVisitCount12Months,EDVisitCount3Months,EDVisitCount12Months,CanopyProduct,PlanID,sequence_num,Source_File,load_time) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"


#cursor.execute("truncate table [Canopy].[ARCH_ADT_ANDOR_ED_input]")

def check_null(column_value):
    if(pd.isnull(column_value)):
        column_value = None
    else:
        column_value
    return column_value

insert_batch = []
rec = ""
counter = 0;
batchnum = 1;

data_len = len(df)


for index,row in df.iterrows():

    rec = (check_null(row['ExternalVisitId']),check_null(row['MemberID']),row['FirstName'],row['LastName'],row['Gender'],row['DOB'],check_null(row['PhoneNumber']),check_null(row['LSPhoneNumber']),row['MedicalGroup'],check_null(row['AdmitDate']),check_null(row['DischargeDate']),row['PatientClass'],check_null(row['EncounterType']),row['SiteName'],row['LocationName'],check_null(row['Diagnosis']),row['AdmittedInpatient'],None,None,row['EDVisitCount3Months'],row['EDVisitCount12Months'],row['CanopyProduct'],row['PlanID'],row['sequence_num'],row['Source_File'],row['load_time'])    
    insert_batch.append(rec)
    counter = counter + 1;
    if(counter%100 == 0):
        cursor.executemany(qry_insert,insert_batch)
        cursor.commit()
        counter = 0
        data_len = data_len - len(insert_batch)
        insert_batch = []
        batchnum = batchnum + 1

if data_len != 0:
    cursor.executemany(qry_insert,insert_batch)  

    cursor.execute(qry_insert,rec)
    
print("Data Load Done")

cnxn.commit()
cursor.close()

Data Load Done
